In [2]:
import pandas as p


In [43]:
f = p.read_csv("testSeperate.csv")
# ft = p.read_csv("test2.csv")

In [40]:
print(f)

     ax0    ay0    az0     gx0     gy0     gz0    ax1    ay1    az1     gx1  \
0   0.17   0.09   0.63  -19.75    5.69   10.94  -0.29  -2.00   7.84  -32.13   
1  -2.98   0.54   5.10 -228.94  -11.12  128.19  -5.71  -6.47  -0.56  -77.00   
2   2.61   1.62   6.09 -125.19   26.56   44.88  -0.87   4.47  10.03 -245.25   
3   1.61   0.57   2.74  -53.25   41.44   33.56  -1.30  -2.26   5.44 -209.56   
4   2.51   1.22   1.98  -61.94   13.13   54.38   2.81  -1.63   5.53 -203.56   
5   2.14   2.55   4.40 -119.25   17.31  122.44  -4.78  -7.19   3.29 -116.37   
6  -0.26   0.30   0.26  -13.38   -0.88    3.44   0.03   0.87   0.64  -22.00   
7   1.63   2.52   7.23 -180.87  102.56  219.56 -10.20  -2.59   4.42   27.06   
8   0.11   2.96   0.13  -55.63  -10.44   27.19  -2.73   2.62   7.12 -220.19   
9  -0.22   3.96   1.61 -120.19    2.19   62.06  -8.88  -8.05   3.86 -201.94   
10 -0.77   3.47   2.73  -77.37  -47.69   49.75  -2.31   0.52   6.01 -147.00   
11  0.28   0.31  -0.39    4.94    6.56    5.12   0.5

In [44]:
finalF= p.DataFrame()
label = f.iloc[:,-1]
for i in range(6):
    
    curF = f.iloc[:,i::6]
    std = curF.std(1)
    kur = curF.kurtosis(1)
    skew = curF.skew(1)
    q25 = curF.quantile(0.25,1)
    q50 = curF.quantile(0.5,1)
    q75 = curF.quantile(0.75,1)
    curdF = p.concat([std.rename("std%s"%i), kur.rename("kur%s"%i),skew.rename("skew%s"%i),q25.rename("q25%s"%i),q50.rename("q50%s"%i),q75.rename("q75%s"%i)],axis=1)
    finalF = p.concat([finalF,curdF],axis = 1)
finalF = p.concat([finalF,label],axis = 1)
print(finalF)
finalF.to_csv("extractedFeatures.csv", sep=',',encoding="utf-8")

        std0       kur0     skew0   q250  q500   q750      std1       kur1  \
0   2.621924   7.720191 -2.581741 -1.695 -0.20 -0.050  1.607699   4.167047   
1   4.136609   4.967296 -2.469237 -0.415 -0.22 -0.050  2.462913   1.519555   
2   1.849559   3.478855 -1.846658 -1.300 -0.06  0.055  1.769865   1.917477   
3   4.756320   9.789992 -3.093862 -0.220  0.23  0.285  3.239655   7.049175   
4   2.345078   3.109249 -1.748536 -0.095  0.12  0.235  2.519789   5.539929   
5   1.006265   2.720823 -1.645840 -0.265  0.09  0.180  2.893256   7.878354   
6   2.479070   2.921396 -1.970776 -0.360 -0.06  0.205  2.035203   5.534006   
7   3.804135   6.286645  0.245908 -0.400 -0.09  0.120  2.110050   6.679039   
8   2.978685   6.264411 -2.519326 -0.575 -0.17  0.040  6.243926  14.013501   
9   1.973170   8.602084 -2.767097 -0.435 -0.04  0.095  2.744732   5.996956   
10  5.572545   4.576972 -2.320215 -0.210  0.12  0.305  2.267441   6.921836   
11  4.173669   3.625317 -2.090684 -0.625  0.18  0.485  1.455958 

In [24]:
import numpy as np

def pandas2arff(df,filename,wekaname = "pandasdata",cleanstringdata=True,cleannan=True):
    """
    converts the pandas dataframe to a weka compatible file
    df: dataframe in pandas format
    filename: the filename you want the weka compatible file to be in
    wekaname: the name you want to give to the weka dataset (this will be visible to you when you open it in Weka)
    cleanstringdata: clean up data which may have spaces and replace with "_", special characters etc which seem to annoy Weka. 
                     To suppress this, set this to False
    cleannan: replaces all nan values with "?" which is Weka's standard for missing values. 
              To suppress this, set this to False
    """
    import re
    
    def cleanstring(s):
        if s!="?":
            return re.sub('[^A-Za-z0-9]+', "_", str(s))
        else:
            return "?"
            
    dfcopy = df #all cleaning operations get done on this copy

    
    if cleannan!=False:
        dfcopy = dfcopy.fillna(-999999999) #this is so that we can swap this out for "?"
        #this makes sure that certain numerical columns with missing values don't get stuck with "object" type
 
    f = open(filename,"w")
    arffList = []
    arffList.append("@relation " + wekaname + "\n")
    #look at each column's dtype. If it's an "object", make it "nominal" under Weka for now (can be changed in source for dates.. etc)
    for i in range(df.shape[1]):
        if dfcopy.dtypes[i]=='O' or (df.columns[i] in ["Class","CLASS","class"]):
            if cleannan!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].replace(to_replace=-999999999, value="?")
            if cleanstringdata!=False:
                dfcopy.iloc[:,i] = dfcopy.iloc[:,i].apply(cleanstring)
            _uniqueNominalVals = [str(_i) for _i in np.unique(dfcopy.iloc[:,i])]
            _uniqueNominalVals = ",".join(_uniqueNominalVals)
            _uniqueNominalVals = _uniqueNominalVals.replace("[","")
            _uniqueNominalVals = _uniqueNominalVals.replace("]","")
            _uniqueValuesString = "{" + _uniqueNominalVals +"}" 
            arffList.append("@attribute " + df.columns[i] + _uniqueValuesString + "\n")
        else:
            arffList.append("@attribute " + df.columns[i] + " real\n") 
            #even if it is an integer, let's just deal with it as a real number for now
    arffList.append("@data\n")           
    for i in range(dfcopy.shape[0]):#instances
        _instanceString = ""
        for j in range(df.shape[1]):#features
                if dfcopy.dtypes[j]=='O':
                    _instanceString+="\"" + str(dfcopy.iloc[i,j]) + "\""
                else:
                    _instanceString+=str(dfcopy.iloc[i,j])
                if j!=dfcopy.shape[1]-1:#if it's not the last feature, add a comma
                    _instanceString+=","
        _instanceString+="\n"
        if cleannan!=False:
            _instanceString = _instanceString.replace("-999999999.0","?") #for numeric missing values
            _instanceString = _instanceString.replace("\"?\"","?") #for categorical missing values
        arffList.append(_instanceString)
    f.writelines(arffList)
    f.close()
    del dfcopy
    return True

In [45]:
pandas2arff(finalF,"test6.arff")

True